In [33]:
import torch
from torch import nn
from torchmetrics import F1Score, AUROC, Precision, Recall
from torch.utils.data import DataLoader, WeightedRandomSampler

from utils.torch_classes import ECG_Dataset, EarlyStopping

from utils.rnn_models import ECG_LSTM_Classifier, ECG_GRU_Classifier

from utils.train import train_and_eval_model, val_loop
from utils.logging import log_to_json, log_to_csv

from utils.data import calculate_class_weights, calculate_sample_weights

import numpy as np
from sklearn.model_selection import ParameterSampler, StratifiedKFold

import random
import time

**TRAINING MODELS**

-- Setting Seed

In [10]:
torch.manual_seed(42)
np.random.seed(42)
random.seed(42)
torch.backends.cudnn.deterministic = True

-- Loading Preprocessed **(Train+Val)** Data

In [25]:
# load pre processed data
data_path = "data/ecg_preprocessed_train_val.npz"
data = np.load(data_path)

X = data["X"]
y = data["y"]

# # assign the weights to all of y_train
# train_sample_weights = np.array(class_weights)[y]

# train_sample_weights = torch.from_numpy(train_sample_weights).float()

# weighted_sampler = WeightedRandomSampler(
#     weights= train_sample_weights,
#     num_samples= len(train_sample_weights),
#     replacement=True,
# )

# NUM_CLASSES = len(class_weights)

-- Set up KFold

In [ ]:
K = 5
kfold = StratifiedKFold(n_splits=K, shuffle=True, random_state=42)

-- Set up Param Grid

In [ ]:
EXPERIMENTS = 20

param_grid = {
    "model": ["LSTM", "GRU"],
    "bidirectional": [True],
    "optimizer": ["Adam", "AdamW", "SGD"],
    "momentum": np.linspace(0.9,0.999,3).tolist(),
    "batch_size": [32,64,128],
    "hidden_size": [32,64,128],
    "num_layers": [1,2],
    "dropout": [0.1,0.2,0.3,0.5],
    "weight_decay": np.logspace(-5,-2,num=5).tolist(),
    "learning_rate": np.logspace(-4,-3,num=5).tolist(),
}

configs = list(ParameterSampler(
    param_grid, 
    n_iter=EXPERIMENTS, 
    random_state=42))

In [19]:
# Print 2 configs to visualise what they look like
configs[:1]

[{'weight_decay': 0.0017782794100389228,
  'optimizer': 'AdamW',
  'num_layers': 1,
  'momentum': 0.9495,
  'model': 'GRU',
  'learning_rate': 0.0001668100537200059,
  'hidden_size': 64,
  'dropout': 0.5,
  'bidirectional': True,
  'batch_size': 128}]

-- Random Search + Stratified K Fold:  
**To find the best model conifguration**

In [ ]:
LOG_FOLDER = "random_search_results"
EPOCHS = 30

PATIENCE = 5
DELTA = 0.0001

NUM_CLASSES = 5

device = "cuda" if torch.cuda.is_available() else "cpu"

results_summary_json = []
results_summary_csv = []

val_precision_metric = Precision(task="multiclass", num_classes=NUM_CLASSES, average="macro").to(device)
val_recall_metric = Recall(task="multiclass", num_classes=NUM_CLASSES, average="macro").to(device)
val_f1_metric = F1Score(task="multiclass", num_classes=NUM_CLASSES, average="macro").to(device)
val_auc_metric = AUROC(task="multiclass", num_classes=NUM_CLASSES, average="macro").to(device)


# Random Search Configs
for i, params in enumerate(configs):

    print(f"-------------- Experiment {i + 1}/{len(configs)} ----------------")
    try: 
        
        BATCH_SIZE = params['batch_size']
        OPTIM = params['optimizer']

        INPUT_SIZE = 1
        HIDDEN_SIZE = params["hidden_size"]
        NUM_LAYERS = params["num_layers"]
        DROPOUT = params["dropout"]
        BIDIRECTIONAL = params["bidirectional"]

        MOMENTUM = params["momentum"]
        LEARNING_RATE = params["learning_rate"]
        WEIGHT_DECAY = params["weight_decay"]

        MODEL = params['model']

        fold_metrics = []

        print(f"Model: {'Bi' if BIDIRECTIONAL else ''}{MODEL}")

        # Loop over Each Fold
        for fold, (train_index, val_index) in enumerate(kfold.split(X,y)):

            # Reset Metrics
            for metric in [val_precision_metric,
                        val_recall_metric,
                        val_f1_metric,
                        val_auc_metric]:
            
                metric.reset()

            print(f"\n---------Fold {fold+1}/{K} ---------\n")

            # Split into train and val subsets
            X_train_fold, y_train_fold = X[train_index], y[train_index]
            X_val_fold, y_val_fold = X[val_index], y[val_index]

            train_dataset = ECG_Dataset(X_train_fold, y_train_fold)
            val_dataset = ECG_Dataset(X_val_fold,y_val_fold)

            # Calculate the class weights 
            labels, class_weights = calculate_class_weights(y_train_fold)
            sample_weights = np.array(class_weights)[y_train_fold]

            # Create Weighted Random Sampler
            weighted_sampler = WeightedRandomSampler(weights=sample_weights,
                                            num_samples=len(sample_weights),
                                            replacement=True)

            # We need to recreate the data loaders 
            train_dataloader = DataLoader(
                dataset=train_dataset,
                batch_size=BATCH_SIZE,
                sampler= weighted_sampler, # this is the Random Weighted Sampler we had created earlier 
                shuffle= False # we dont need to shuffle because we are using the sampler
            )

            val_dataloader = DataLoader(
                dataset=val_dataset,
                batch_size=BATCH_SIZE,
                shuffle=False
            )

            # Create the model
            if MODEL == "LSTM":
                model = ECG_LSTM_Classifier(
                    input_size=INPUT_SIZE,
                    hidden_size=HIDDEN_SIZE,
                    num_layers=NUM_LAYERS,
                    num_classes=NUM_CLASSES,
                    dropout=DROPOUT,
                    bidirectional=BIDIRECTIONAL
                )

            elif MODEL == "GRU":
                model = ECG_GRU_Classifier(
                    input_size=INPUT_SIZE,
                    hidden_size=HIDDEN_SIZE,
                    num_layers=NUM_LAYERS,
                    num_classes=NUM_CLASSES,
                    dropout=DROPOUT,
                    bidirectional=BIDIRECTIONAL
                )

            model.to(device)

            # Create the optimiers 
            if OPTIM == "Adam":
                optim = torch.optim.Adam(params=model.parameters(),
                                        lr=LEARNING_RATE,
                                        weight_decay=WEIGHT_DECAY)

            elif OPTIM == "AdamW":
                optim = torch.optim.AdamW(params=model.parameters(),
                                        lr=LEARNING_RATE,
                                        weight_decay=WEIGHT_DECAY)
                
            elif OPTIM == "SGD":
                optim = torch.optim.SGD(params=model.parameters(),
                                        lr=LEARNING_RATE,
                                        momentum=MOMENTUM,
                                        weight_decay=WEIGHT_DECAY)

            else: 
                raise ValueError(f"Unknown optimizer: {OPTIM}")
            
            # Create Loss function
            loss_func = nn.CrossEntropyLoss()

            # Create Early Stopper
            early_stopper = EarlyStopping(
                    patience=PATIENCE, 
                    delta=DELTA,
                    checkpoint_path=f"{LOG_FOLDER}/checkpoints/experiment_{i+1}/fold_{fold+1}.pt",
                    verbose=False)
            
            # Train the model 
            start = time.time()
            results = train_and_eval_model(
                model=model, 
                loss_fn=loss_func,
                optimizer=optim,
                device=device,

                train_dataloader=train_dataloader,
                val_dataloader=val_dataloader,
                
                epochs=EPOCHS, # small epoch number during this phase
                early_stopper= early_stopper,

                verbose=False,
                debug=False,

                grad_clip=True,
                max_norm=1.0
            )
            end = time.time()

            epochs_run = len(results["train_loss"])

            total_time = (end-start)
            time_per_epoch = total_time/epochs_run if epochs_run > 0 else 0 # (i+1) is the current epoch

            # Evaluate the best checkpoint
            model.load_state_dict(torch.load(early_stopper.checkpoint_path, map_location=device)["model_state_dict"])
            val_data = val_loop(model=model, 
                                    val_dataloader=val_dataloader,
                                    loss_fn=loss_func, 
                                    device=device)
            
            val_pred = torch.cat(val_data["y_pred"])
            val_true = torch.cat(val_data["y_true"])

            val_pred_logits = torch.cat(val_data["y_pred_logits"])

            # Calculate Precision, Recall, F1 and AUC
            fold_precision = val_precision_metric(val_pred.to(device), val_true.to(device)).item()
            fold_recall = val_recall_metric(val_pred.to(device), val_true.to(device)).item()
            fold_f1 = val_f1_metric(val_pred.to(device), val_true.to(device)).item()
            fold_auc = val_auc_metric(val_pred_logits.to(device), val_true.to(device)).item()

            fold_metrics.append({
                "fold": fold+1,
                "precision": fold_precision,
                "recall": fold_recall,
                "f1": fold_f1,
                "auc": fold_auc,
                "time_per_epoch": time_per_epoch, 
            })

            print(f"Fold {fold+1}: F1={fold_f1:.3f} | AUC={fold_auc:.3f}")

        # Agreggrate metrics across folds
        avg_recall = np.mean([m["recall"] for m in fold_metrics])
        avg_precision = np.mean([m["precision"] for m in fold_metrics])
        avg_f1 = np.mean([m["f1"] for m in fold_metrics])
        avg_auc = np.mean([m["auc"] for m in fold_metrics])

        avg_time_per_epoch = np.mean(m["time_per_epoch"] for m in fold_metrics)

        results_summary_csv.append({
            "experiment": i+1,
            "avg_precision": avg_precision,
            "avg_recall": avg_recall,
            "avg_f1": avg_f1,
            "avg_auc": avg_auc,
            "avg_time_per_epoch": avg_time_per_epoch
        })

        if OPTIM != "SGD":
            params["momentum"] = None

        results_summary_json.append({
            "experiment": i+1,
            **params,
            "fold_metrics": fold_metrics
        })

        log_to_csv(f"{LOG_FOLDER}/results.csv", results_summary_csv)
        log_to_json(f"{LOG_FOLDER}/results.json", results_summary_json)

        print(f"\n Experiment {i+1} Done: Avg F1={avg_f1:.3f}, Avg AUC={avg_auc:.3f}\n")

    except Exception as e:
        print(f"Experiment {i+1} failed: {e}")

print("\n---------------- All Experiments Completed ----------------")



---------Fold 1/5 ---------


---------Fold 2/5 ---------


---------Fold 3/5 ---------


---------Fold 4/5 ---------


---------Fold 5/5 ---------



In [37]:
fold_metrics = [
    {"f1": 0.11},
    {"f1": 0.12},
    {"f1": 0.13},
]

np.mean([m["f1"] for m in fold_metrics])

np.float64(0.12)